In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import os

import warnings
warnings.filterwarnings('ignore')

In [13]:
%%time
dtype = {
    'userID': 'int16',
    'answerCode': 'int8',
    'KnowledgeTag': 'int16'
}   

# data set load
DATA_PATH = '/opt/ml/input/data'
train_path = os.path.join(DATA_PATH, 'train_data.csv')
test_path = os.path.join(DATA_PATH, 'test_data.csv')

train_df = pd.read_csv(train_path, dtype=dtype, parse_dates=['Timestamp'])
test_df = pd.read_csv(test_path, dtype=dtype, parse_dates=['Timestamp'])

CPU times: user 2.74 s, sys: 128 ms, total: 2.87 s
Wall time: 2.87 s


In [47]:
train_df['train'] = 1
test_df['train'] = 0

In [48]:
#문제 풀이 시간 feature 'elapsed time'

# train data에 elapsed time 추가
diff_train = train_df.loc[:, ['userID', 'Timestamp']].groupby('userID').diff().shift(-1)
diff_train = diff_train['Timestamp'].apply(lambda x: x.total_seconds())
train_df['elapsed'] = diff_train

# test data에 elapsed time 추가
diff_test = test_df.loc[:, ['userID', 'Timestamp']].groupby('userID').diff().shift(-1)
diff_test = diff_test['Timestamp'].apply(lambda x: x.total_seconds())
test_df['elapsed'] = diff_test

# answerCode가 -1인 data 제거 (예측해야 하는 값)
test_df = test_df[test_df['answerCode'] != -1]
test_df = test_df.sort_values(by=['userID', 'Timestamp']).reset_index(drop=True)

df = pd.concat([train_df, test_df])

In [49]:
import pytz

# 한국 기준으로 Timestamp 변경
kst = pytz.timezone('Asia/Seoul')
df['Timestamp'] = df['Timestamp'].dt.tz_localize('UTC').dt.tz_convert(kst)

In [51]:
# testID의 대분류 featured인 'testCode' 

df['testCode'] = df['testId'].apply(lambda x : x[1:4])

#testCode를 1~9의 정수형 값으로 변환
df['testCode'] = df['testCode'].astype('int') //10

In [52]:
# testCode를 다시 크게 범주화한 'bigtestCode'

def big_testcode_map(x:int) -> int:
    x = (x-1) // 3
    return x

df['bigtestCode'] = df['testCode'].apply(big_testcode_map)

In [53]:
#user가 과거에 맞춘 문제 비율 'pastCorrectRate'

df['shift'] = df.groupby('userID')['answerCode'].shift().fillna(0)
df['pastCorrect'] = df.groupby('userID')['shift'].cumsum()
df['pastCount'] = df.groupby('userID').cumcount()
df['pastCorrectRate'] = (df['pastCorrect'] / df['pastCount']).fillna(0)
df.drop(['shift', 'pastCount'], axis=1, inplace=True)

In [54]:
#user의 test별 평균 정답률 'testMeanCorrect'

df_mean = df.copy()
group = df.groupby(['userID', 'testId'])['answerCode'].mean() #group = series

df_mean['testMeanCorrect'] = 0

df_mean = df_mean.sort_values(by=['userID', 'testId']).reset_index(drop=True)

In [55]:
idx_lst = list(group.index)
i = 0 #idx_lst의 인덱싱을 위한 변수
j = 0 #df_mean의 row 인덱싱을 위한 변수

while (i < len(idx_lst)):
    user, test = idx_lst[i]
    while (df_mean.iloc[j]['userID'] == user) & (df_mean.iloc[j]['testId'] == test):
        df_mean.loc[j, 'testMeanCorrect'] = group[user, test]
        j += 1
        if j == len(df_mean):
            break
    i += 1

In [56]:
df_mean = df_mean.sort_values(by=['userID', 'Timestamp']).reset_index(drop=True)
df_mean

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,elapsed,train,testCode,bigtestCode,pastCorrect,pastCorrectRate,testMeanCorrect
0,0,A060001001,A060000001,1,2020-03-24 09:17:11+09:00,7224,3.0,1,6,1,0.0,0.000000,1.0
1,0,A060001002,A060000001,1,2020-03-24 09:17:14+09:00,7225,8.0,1,6,1,1.0,1.000000,1.0
2,0,A060001003,A060000001,1,2020-03-24 09:17:22+09:00,7225,7.0,1,6,1,2.0,1.000000,1.0
3,0,A060001004,A060000001,1,2020-03-24 09:17:29+09:00,7225,7.0,1,6,1,3.0,1.000000,1.0
4,0,A060001005,A060000001,1,2020-03-24 09:17:36+09:00,7225,11.0,1,6,1,4.0,1.000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2525951,7441,A030071005,A030000071,0,2020-06-05 15:50:21+09:00,438,6632178.0,1,3,0,1.0,0.250000,0.2
2525952,7441,A040165001,A040000165,1,2020-08-21 10:06:39+09:00,8836,11.0,1,4,1,1.0,0.200000,1.0
2525953,7441,A040165002,A040000165,1,2020-08-21 10:06:50+09:00,8836,46.0,1,4,1,2.0,0.333333,1.0
2525954,7441,A040165003,A040000165,1,2020-08-21 10:07:36+09:00,8836,73.0,1,4,1,3.0,0.428571,1.0


In [57]:
#user의 test 별 정답률 추세

df_mean['shift'] = df_mean.groupby(['userID', 'testId'])['answerCode'].shift().fillna(0)
df_mean['test_past'] = df_mean.groupby(['userID', 'testId'])['shift'].cumsum()
df_mean['test_count'] = df_mean.groupby(['userID', 'testId']).cumcount()
df_mean['testCorrectRate'] = (df_mean['test_past'] / df_mean['test_count']).fillna(0)
df_mean.drop(['shift', 'test_past', 'test_count'], axis=1, inplace=True)

In [42]:
# 다시 csv 파일로 만들어서 정리

save_path = os.path.join(DATA_PATH, 'save_data.csv')
df_mean.to_csv(save_path)

In [59]:
#test랑 train data 분리해서 csv 파일로 만들기

save_train = df_mean.loc[df_mean['train']]
save_train.drop(['train'], axis=1, inplace=True)
save_test = df_mean.loc[df_mean['train'] == 0]
save_test.drop(['train'], axis=1, inplace=True)

train_save_path = os.path.join(DATA_PATH, 'save_train.csv')
save_train.to_csv(train_save_path, index=False)
test_save_path = os.path.join(DATA_PATH, 'save_test.csv')
save_test.to_csv(test_save_path, index=False)